In [1]:
!nvidia-smi

Wed Apr  1 13:06:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.26       Driver Version: 430.26       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1D:00.0 Off |                  N/A |
| 27%   37C    P0    53W / 250W |      0MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1E:00.0 Off |                  N/A |
| 28%   38C    P0    61W / 250W |      0MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

import numpy as np
import matplotlib.pyplot as plt
import cv2

import sys
sys.path.append('/mnt/disk1/project/SMhospital/capsule/ce_packages')

from ce_utils import load_data, one_hot, test_batch_idxs, batch_flatten
from ce_utils import model_prob
from ce_utils import classification_metric
from ce_model import binary_model, ensemble_model

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

# Load Data

In [3]:
from itertools import product
import glob

def target_preprocessings(phase_a_switch = [1, 1, 1], phase_b_switch = True, mode = 'load'):
    """
    phase_a_switch = [1, 1, 1], [0, 0 ,1], [1, 1, 0].... 
    that means [flip, rotate, blur_sharp]
    """
    phase0 = ['_c']
    phase1 = {1: ['-', 'f'], 0: ['-']}
    phase2 = {1: ['-', 'r1', 'r2', 'r3'], 0: ['-']}
    phase3 = {1: ['-', 'ab', 'mb', 'eh'], 0: ['-']}
    phase4 = ['s_-30_v_30', 's_-30_v_-30', 's_30_v_-30', 's_30_v_30']
    
    if mode == 'load':
        phase_a_items = [phase1[phase_a_switch[0]], phase2[phase_a_switch[1]], phase3[phase_a_switch[2]]]
    elif mode == 'preprocessing':
        phase_a_items = [phase0, phase1[phase_a_switch[0]], phase2[phase_a_switch[1]], phase3[phase_a_switch[2]]]
    
    phase_a = []
    for i in list(product(*phase_a_items)):
        phase_a.append('_'.join(i))

    if not phase_b_switch != True:
        phase_b = []
        for i in list(product(*[phase_a, phase4])):
            phase_b.append('_'.join(i))
        return list(np.hstack([phase_a, phase_b]))
    else:
        return phase_a 

class ce_load_dataset:
    def __init__(self, phase, data, pre_a, pre_b, img_ch = 'bgr', ext_name = True):
        self.phase = phase        # 'train' or 'test'
        self.data = data          # 'sm', 'sm_core', 'sm_v2', 'sm_x160', ...
        self.pre_a = pre_a        # [1, 1, 1], [0, 0 ,1], [1, 1, 0].... 
        self.pre_b = pre_b        # True or False
        self.img_ch = img_ch      # 'bgr', 'rgb', and 'hsv'
        self.ext_name = ext_name  # True or False

    def load_path(self, cls, les, data_dir = '/mnt/disk2/data/private_data/SMhospital/capsule/1 preprocessed'):
        """
        phase = 'train', 'test'
        cls: [les]  
          'n': ['neg']
          'h': ['redspot', 'angio', 'active'], 
          'd': ['ero', 'ulc', 'str'],
          'p': ['amp', 'lym', 'tum']}
        pre_a[0] must be 0
        """
        lesions = dict(neg = 'negative', 
                       redspot = 'red_spot', angio = 'angioectasia', active = 'active_bleeding', 
                       ero = 'erosion', ulcer = 'ulcer', str = 'stricture', 
                       amp = 'ampulla_of_vater', lym = 'lymphoid_follicles', tum = 'small_bowel_tumor')
        classes = dict(n = 'negative', h = 'hemorrhagic', d = 'depressed', p = 'protruded')

        path = os.path.join(data_dir, self.data, self.phase, classes[cls], lesions[les])
        pathlist = glob.glob(path + '/*.jpg')
        if self.pre_b != True:
            path_in_phase = []
            for p in pathlist:
                name = os.path.basename(p)
                if (name.split('c_')[-1])[:-4] in target_preprocessings(self.pre_a, self.pre_b):
                    path_in_phase.append(p)   
            return np.asarray(path_in_phase)
        else:
            return np.asarray(pathlist)

    def load_image_from_path(self, pathlist,image_ch = 'bgr', extract_name = False):
        data = []
        for i in pathlist:
            temp = cv2.imread(i)
            if image_ch == 'bgr':
                pass
            elif image_ch == 'rgb':
                temp = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
            elif image_ch == 'hsv':
                temp = cv2.cvtColor(temp, cv2.COLOR_BGR2HSV)
            data.append(temp)
        if extract_name != False:
            name = []
            for i in pathlist:
                name.append(os.path.basename(i))
            return np.asarray(data), np.asarray(name)
        else:
            return np.asarray(data) 

    def load_data(self, cls, les):
        pathlist = self.load_path(cls, les)
        return  self.load_image_from_path(pathlist, image_ch = self.img_ch, extract_name = self.ext_name)

In [4]:
ce = ce_load_dataset(phase = 'test', data = 'sm_x160_v2', pre_a = [0, 0, 0], pre_b = False)

In [5]:
neg_testset = ce.load_data('n', 'neg')
redspot_testset = ce.load_data('h', 'redspot')
angio_testset = ce.load_data('h', 'angio')
active_testset = ce.load_data('h', 'active')
ero_testset = ce.load_data('d', 'ero')
ulcer_testset = ce.load_data('d', 'ulcer')
str_testset = ce.load_data('d', 'str')

neg_label = one_hot(neg_testset[0], 0)
redspot_label = one_hot(redspot_testset[0], 1)
angio_label = one_hot(angio_testset[0], 1)
active_label = one_hot(active_testset[0], 1)
ero_label = one_hot(ero_testset[0], 1)
ulcer_label = one_hot(ulcer_testset[0], 1)
str_label = one_hot(str_testset[0], 1)

print("negative:",neg_testset[0].shape, neg_label.shape)
print("red spot:",redspot_testset[0].shape, redspot_label.shape)
print("angioectasia:",angio_testset[0].shape, angio_label.shape)
print("active_bleeding:",active_testset[0].shape, active_label.shape)
print("erosion:",ero_testset[0].shape, ero_label.shape)
print("ulcer:",ulcer_testset[0].shape, ulcer_label.shape)
print("stricture:",str_testset[0].shape, str_label.shape)

negative: (1562, 512, 512, 3) (1562, 2)
red spot: (139, 512, 512, 3) (139, 2)
angioectasia: (5, 512, 512, 3) (5, 2)
active_bleeding: (108, 512, 512, 3) (108, 2)
erosion: (185, 512, 512, 3) (185, 2)
ulcer: (183, 512, 512, 3) (183, 2)
stricture: (26, 512, 512, 3) (26, 2)


In [6]:
neg_testset[1]

array(['AJ___02-08-16___1015395__c_-_-_-.jpg',
       'DJ___01-58-32___1014228__c_-_-_-.jpg',
       'DJ___01-59-21___1014325__c_-_-_-.jpg', ...,
       '희정___04-45-38___2047831__c_-_-_-.jpg',
       '희정___05-17-56___2052919__c_-_-_-.jpg',
       '희정___05-19-44___2053248__c_-_-_-.jpg'], dtype='<U37')

# Model Inference

In [7]:
b_list = sorted([i[:-5] for i in os.listdir('./model/X160_v2/comparison') if 'meta' in i])

In [9]:
b_list

['X160_v2_B_2001231929_120_t_acc_0.61_v_acc_0.59_t_loss_0.656259_v_loss_0.651237',
 'X160_v2_B_2001231929_130_t_acc_0.63_v_acc_0.61_t_loss_0.641107_v_loss_0.643231',
 'X160_v2_B_2001231930_140_t_acc_0.65_v_acc_0.64_t_loss_0.622017_v_loss_0.621604',
 'X160_v2_B_2001231930_150_t_acc_0.67_v_acc_0.67_t_loss_0.610996_v_loss_0.599846',
 'X160_v2_B_2001231930_160_t_acc_0.69_v_acc_0.69_t_loss_0.597492_v_loss_0.582524',
 'X160_v2_B_2001231957_10_t_acc_0.64_v_acc_0.48_t_loss_0.687736_v_loss_0.692382',
 'X160_v2_B_2001232006_30_t_acc_0.56_v_acc_0.54_t_loss_0.685987_v_loss_0.688833',
 'X160_v2_B_2001232006_40_t_acc_0.56_v_acc_0.55_t_loss_0.683776_v_loss_0.687903',
 'X160_v2_B_2001232007_50_t_acc_0.58_v_acc_0.58_t_loss_0.679669_v_loss_0.682847',
 'X160_v2_B_2001232007_60_t_acc_0.61_v_acc_0.61_t_loss_0.669189_v_loss_0.673316',
 'X160_v2_B_2001232007_70_t_acc_0.63_v_acc_0.61_t_loss_0.659057_v_loss_0.663516',
 'X160_v2_B_2001232039_1660_t_acc_0.90_v_acc_0.85_t_loss_0.260919_v_loss_0.351091',
 'X160_v2

In [10]:
b_list = {0: 'X160_v2_B_2001232042_1810_t_acc_0.91_v_acc_0.87_t_loss_0.218927_v_loss_0.321115', 
          1: 'X160_v2_f_B_2001232102_3410_t_acc_0.96_v_acc_0.94_t_loss_0.111460_v_loss_0.173293', 
          2: 'X160_v2_r_B_2001232134_4750_t_acc_0.96_v_acc_0.96_t_loss_0.119850_v_loss_0.116892', 
          3: 'X160_v2_b_B_2001232052_2240_t_acc_0.93_v_acc_0.92_t_loss_0.182928_v_loss_0.218141', 
          4: 'X160_v2_sv_B_2001232107_2700_t_acc_0.93_v_acc_0.92_t_loss_0.180744_v_loss_0.204723', 
          5: 'X160_v2_fr_B_2001241254_7940_t_acc_0.98_v_acc_0.96_t_loss_0.062579_v_loss_0.118580'}

In [14]:
h_label = np.vstack([redspot_label, angio_label, active_label])
d_label = np.vstack([ero_label, ulcer_label, str_label])
labels = np.vstack([neg_label, h_label, d_label])

cls = classification_metric()

for i in range(6):
    
    b_path = './model/X160_v2/comparison/' + b_list[i]
    B_model = binary_model(b_path)
    
    print(b_list[i].split('_2001')[0])
    
    neg_b_output = model_prob(B_model, neg_testset[0])
    redspot_b_output = model_prob(B_model, redspot_testset[0])
    angio_b_output = model_prob(B_model, angio_testset[0])
    active_b_output = model_prob(B_model, active_testset[0])
    ero_b_output = model_prob(B_model, ero_testset[0])
    ulcer_b_output = model_prob(B_model, ulcer_testset[0])
    str_b_output  = model_prob(B_model, str_testset[0])
    
    neg_b_acc = cls.accuracy(neg_label, neg_b_output, 'negative')
    redspot_b_acc = cls.accuracy(redspot_label, redspot_b_output, 'red spot')
    angio_b_acc = cls.accuracy(angio_label, angio_b_output, 'angioectasia')
    active_b_acc = cls.accuracy(active_label, active_b_output, 'active bleeding')
    ero_b_acc = cls.accuracy(ero_label, ero_b_output, 'erosion')
    ulcer_b_acc = cls.accuracy(ulcer_label, ulcer_b_output, 'ulcer')
    str_b_acc = cls.accuracy(str_label, str_b_output, 'stricture')
    
    h_b_output = np.vstack([redspot_b_output, angio_b_output, active_b_output])
    d_b_output = np.vstack([ero_b_output, ulcer_b_output, str_b_output])
    b_outputs = np.vstack([neg_b_output, h_b_output, d_b_output])
    
    total_acc = cls.accuracy(labels, b_outputs, 'total')
    print('')

X160_v2_B
#: 1562 / Inference Time: 00:00:08
#: 139 / Inference Time: 00:00:00
#: 5 / Inference Time: 00:00:00
#: 108 / Inference Time: 00:00:00
#: 185 / Inference Time: 00:00:01
#: 183 / Inference Time: 00:00:00
#: 26 / Inference Time: 00:00:00
negative: 87.84 %
red spot: 56.12 %
angioectasia: 40.00 %
active bleeding: 95.37 %
erosion: 87.03 %
ulcer: 89.62 %
stricture: 88.46 %
total: 86.19 %

X160_v2_f_B
#: 1562 / Inference Time: 00:00:08
#: 139 / Inference Time: 00:00:00
#: 5 / Inference Time: 00:00:00
#: 108 / Inference Time: 00:00:00
#: 185 / Inference Time: 00:00:00
#: 183 / Inference Time: 00:00:00
#: 26 / Inference Time: 00:00:00
negative: 96.67 %
red spot: 79.14 %
angioectasia: 100.00 %
active bleeding: 99.07 %
erosion: 94.59 %
ulcer: 88.52 %
stricture: 88.46 %
total: 94.75 %

X160_v2_r_B
#: 1562 / Inference Time: 00:00:08
#: 139 / Inference Time: 00:00:00
#: 5 / Inference Time: 00:00:00
#: 108 / Inference Time: 00:00:00
#: 185 / Inference Time: 00:00:01
#: 183 / Inference Time: